<a href="https://colab.research.google.com/github/forest1988/colaboratory/blob/main/prophetnet_seq2seqtrainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
torch.__version__

'1.7.0+cu101'

In [3]:
!git clone https://github.com/huggingface/transformers.git
#git clone https://github.com/forest1988/transformers.git

fatal: destination path 'transformers' already exists and is not an empty directory.


In [4]:
%cd transformers/

/content/transformers


In [5]:
!pip install -e .

Obtaining file:///content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Found existing installation: transformers 3.4.0
    Can't uninstall 'transformers'. No files were found to uninstall.
  Running setup.py develop for transformers


In [6]:
%cd /content/transformers/examples/seq2seq/

/content/transformers/examples/seq2seq


In [7]:
# to avoid the "ModuleNotFoundError" 
import sys
print(sys.path)
sys.path.append('/content/transformers/src')
print(sys.path)

['', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/content/transformers_modified/src', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython']
['', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/content/transformers_modified/src', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython', '/content/transformers/src']


In [8]:
# If "ModuleNotFoundError" occurs in this cell, please re-run the runtime. 
# (The first time Colab runtime runs, it seems the sys.path is not correctly updated when we use `pip install -e .`)

import transformers

In [9]:
transformers.__version__

'3.4.0'

## dataset 

In [10]:
!wget https://cdn-datasets.huggingface.co/summarization/xsum.tar.gz
!tar -xzvf xsum.tar.gz
# !export XSUM_DIR=${PWD}/xsum
# %env XSUM_DIR=${PWD}/xsum

--2020-11-05 17:52:07--  https://cdn-datasets.huggingface.co/summarization/xsum.tar.gz
Resolving cdn-datasets.huggingface.co (cdn-datasets.huggingface.co)... 99.86.34.36, 99.86.34.88, 99.86.34.110, ...
Connecting to cdn-datasets.huggingface.co (cdn-datasets.huggingface.co)|99.86.34.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 204844092 (195M) [application/x-tar]
Saving to: ‘xsum.tar.gz.1’

xsum.tar.gz.1       100%[===================>] 195.35M   102MB/s    in 1.9s    

2020-11-05 17:52:09 (102 MB/s) - ‘xsum.tar.gz.1’ saved [204844092/204844092]

xsum/
xsum/train.target
xsum/train.source
xsum/val.source
xsum/val.target
xsum/test.source
xsum/test.target


In [11]:
import os
os.environ['XSUM_DIR']=os.path.join(os.getcwd(), 'xsum')

In [12]:
!echo $XSUM_DIR

/content/transformers/examples/seq2seq/xsum


In [13]:
!ls $XSUM_DIR

test.source  test.target  train.source	train.target  val.source  val.target


### Run BART-base

In [14]:
%%capture
!pip install gitpython
!pip install rouge_score
!pip install sacrebleu

In [15]:
!python finetune_trainer.py \
    --learning_rate=3e-5 \
    --do_train --do_eval --evaluate_during_training \
    --predict_with_generate \
    --n_train 300 \
    --n_val 10 \
    --model_name_or_path facebook/bart-base \
    --data_dir $XSUM_DIR \
    --output_dir bart-base-tmp \
    --overwrite_output_dir 

2020-11-05 17:52:28.015292: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/content/transformers/src/transformers/training_args.py:347: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,
11/05/2020 17:52:30 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
11/05/2020 17:52:30 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='bart-base-tmp', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=True, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_r

In [16]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## Run with ProphetNet (Error occurs) 

In [17]:
from transformers import ProphetNetTokenizer, ProphetNetForConditionalGeneration #, ProphetNetForCausalLM

# --- commeented out to avoid CUDA memory error --- 

# tokenizer = ProphetNetTokenizer.from_pretrained('microsoft/prophetnet-large-uncased')
# model = ProphetNetForConditionalGeneration.from_pretrained('microsoft/prophetnet-large-uncased')

# input_ids = tokenizer("Studies have been shown that owning a dog is good for you", return_tensors="pt").input_ids  # Batch size 1
# decoder_input_ids = tokenizer("Studies show that", return_tensors="pt").input_ids  # Batch size 1
# outputs = model(input_ids=input_ids, decoder_input_ids=decoder_input_ids, return_dict=True)

# logits_next_token = outputs.logits  # logits to predict next token as usual
# logits_ngram_next_tokens = outputs.logits_ngram  # logits to predict 2nd, 3rd, ... next tokens

In [18]:
# test how tokeinzer.decoder works 

# print(tokenizer.decode(input_ids[0]))
# print(tokenizer.decode(input_ids[0], skip_special_tokens=True))

In [19]:
# show the style of the output

# print(type(outputs))
# outputs.keys()

In [20]:
# outputs.logits.shape, outputs.logits_ngram.shape

In [21]:
# out_generate = model.generate(input_ids, num_beams=4, max_length=20, early_stopping=True)

In [22]:
# out_generate.shape, out_generate[0]

In [23]:
# tokenizer.decode(out_generate.shape[0], skip_special_tokens=True)

In [24]:
!python finetune_trainer.py \
    --learning_rate=3e-5 \
    --do_train --do_eval --evaluate_during_training \
    --predict_with_generate \
    --n_train 100 \
    --n_val 10 \
    --model_name_or_path microsoft/prophetnet-large-uncased \
    --data_dir $XSUM_DIR \
    --output_dir tmp \
    --overwrite_output_dir

2020-11-05 17:53:24.056751: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/content/transformers/src/transformers/training_args.py:347: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,
11/05/2020 17:53:31 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
11/05/2020 17:53:31 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='tmp', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=True, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=3e-05,

### Refer to T5Tokenizer

``` python
    @add_start_docstrings(PREPARE_SEQ2SEQ_BATCH_DOCSTRING)
    def prepare_seq2seq_batch(
        self,
        src_texts: List[str],
        tgt_texts: Optional[List[str]] = None,
        max_length: Optional[int] = None,
        max_target_length: Optional[int] = None,
        padding: str = "longest",
        return_tensors: str = None,
        truncation: bool = True,
        **kwargs,
    ) -> BatchEncoding:
        if max_length is None:
            max_length = self.max_len
        model_inputs = self(
            src_texts,
            add_special_tokens=True,
            return_tensors=return_tensors,
            max_length=max_length,
            padding=padding,
            truncation=truncation,
            **kwargs,
        )
        if tgt_texts is None:
            return model_inputs
        # Process tgt_texts
        if max_target_length is None:
            max_target_length = max_length
        labels_and_decoder_mask = self(
            tgt_texts,
            add_special_tokens=True,
            return_tensors=return_tensors,
            padding=padding,
            max_length=max_target_length,
            truncation=truncation,
            **kwargs,
        )
        model_inputs["labels"] = labels_and_decoder_mask["input_ids"]
        return model_inputs
```

## Try modified version of ProphetNet Tokenizer

In [34]:
%cd /content/

/content


In [35]:
!ls

drive  sample_data  transformers  transformers_modified


In [36]:
import shutil

if os.path.exists("transformers_modified"):
  print("The repository is already cloned. Remove and re-clone it.")
  shutil.rmtree("transformers_modified")

The repository is already cloned. Remove and re-clone it.


In [37]:
!git clone -b forest1988-prophetnet-prepare-seq2seq-batch https://github.com/forest1988/transformers.git transformers_modified

Cloning into 'transformers_modified'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 49007 (delta 8), reused 8 (delta 3), pack-reused 48988
Receiving objects: 100% (49007/49007), 36.38 MiB | 26.61 MiB/s, done.
Resolving deltas: 100% (34170/34170), done.


In [38]:
%cd transformers_modified/

/content/transformers_modified


In [39]:
!pip install -e . --upgrade

Obtaining file:///content/transformers_modified
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Found existing installation: transformers 3.4.0
    Can't uninstall 'transformers'. No files were found to uninstall.
  Running setup.py develop for transformers


In [40]:
%cd /content/transformers_modified/examples/seq2seq/

# Use already downloaded XSUM in /content/transformers/examples/seq2seq/xsum/

/content/transformers_modified/examples/seq2seq


In [43]:
!CUDA_LAUNCH_BLOCKING=1 python finetune_trainer.py \
    --learning_rate=3e-5 \
    --do_train --do_eval --evaluate_during_training \
    --max_source_length 20 \
    --max_target_length 20 \
    --predict_with_generate \
    --n_train 100 \
    --n_val 10 \
    --model_name_or_path microsoft/prophetnet-large-uncased \
    --data_dir $XSUM_DIR \
    --output_dir tmp \
    --overwrite_output_dir

2020-11-05 17:59:56.159587: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/content/transformers_modified/src/transformers/training_args.py:347: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,
11/05/2020 17:59:58 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
11/05/2020 17:59:58 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='tmp', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=True, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_ra

The warning messages saying `Keyword arguments {'add_prefix_space': False} not recognized.` are relevant to the pull requests below?

- https://github.com/huggingface/transformers/pull/7470
- https://github.com/huggingface/transformers/pull/8329